In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/file/d/1ugeBtBckwyxNbr1UtfxSOdY7fA-KZa4w/view?usp=sharing' 

In [ ]:
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Survey_Results_Dec.csv') 
 
df = pd.read_csv('Survey_Results_Dec.csv')
print(df)

                                            StartDate  \
0                                          Start Date   
1   {"ImportId":"startDate","timeZone":"America/De...   
2                                 2022-04-19 04:25:25   
3                                 2022-04-19 04:25:44   
4                                 2022-04-20 05:33:47   
5                                 2022-04-24 10:17:37   
6                                 2022-04-25 08:08:10   
7                                 2022-04-25 08:04:55   
8                                 2022-04-25 10:17:41   
9                                 2022-04-25 14:28:28   
10                                2022-04-21 06:07:42   
11                                2022-04-29 10:17:14   
12                                2022-04-29 10:19:30   
13                                2022-04-29 11:02:56   
14                                2022-04-29 11:01:44   
15                                2022-04-30 03:15:41   
16                             

In [ ]:
#remove missing values (empty survey responses)
df = df.dropna(subset=['Q100'])

In [ ]:
#df['Q1']

results = df.loc[:, df.columns.str.startswith('Q')]

In [ ]:
results = results.drop([1])

In [ ]:
#switch rows as columns and columns as rows for fleiss kappa formula
results_transposed = results.transpose

In [ ]:
results_transposed

<bound method DataFrame.transpose of                                                    Q1  \
0   "Tucker Carlson seem to have forgotten that Ru...   
5                                                   3   
6                                                   3   
7                                                   3   
8                                                   3   
9                                                   3   
11                                                  3   
12                                                  1   
13                                                  3   
14                                                  3   
15                                                  3   

                                                   Q2  \
0   "Analysis: Would Putin get away with invading ...   
5                                                   2   
6                                                   3   
7                                                 

In [ ]:
results_num = results.drop([0])

In [ ]:
results_T = results_num.transpose()

In [ ]:
results_T

,5,6,7,8,9,11,12,13,14,15
Q1,3,3,3,3,3,3,1,3,3,3
Q2,2,3,3,2,2,3,3,3,3,2
Q3,2,2,2,2,2,2,2,3,3,2
Q4,2,3,3,2,2,3,2,3,3,2
Q5,3,3,3,3,3,3,1,3,3,3
...,...,...,...,...,...,...,...,...,...,...
Q96,3,3,3,2,3,3,2,3,3,3
Q97,3,3,2,2,3,3,3,3,3,3
Q98,2,2,2,2,2,3,2,2,2,2
Q99,3,3,2,2,2,3,3,2,3,2


In [ ]:
results_T.iloc[101].value_counts(normalize=True)['2']

0.6

In [ ]:
#Count the 1, 2, and 3s per row per question
results_P = (results_T.apply(lambda x: x.value_counts(normalize=True), axis=1)
         .fillna(0)
         .astype(float))

In [ ]:
results_P

,1,2,3
Q1,0.1,0.0,0.9
Q2,0.0,0.4,0.6
Q3,0.0,0.8,0.2
Q4,0.0,0.5,0.5
Q5,0.1,0.0,0.9
...,...,...,...
Q96,0.0,0.2,0.8
Q97,0.0,0.2,0.8
Q98,0.0,0.9,0.1
Q99,0.0,0.5,0.5


In [ ]:
conditions = [
    (results_P['1'] > results_P['2']) & (results_P['1'] > results_P['3']),
    (results_P['2'] > results_P['1']) & (results_P['2'] > results_P['3']),
    (results_P['3'] > results_P['1']) & (results_P['3'] > results_P['2']),
    (results_P['1'] == results_P['2']),
    (results_P['2'] == results_P['3']),
    (results_P['1'] == results_P['3'])
]

choices = ['positive','neutral','negative', 'undecided', 'undecided', 'undecided']
results_P['Label'] = np.select(conditions, choices, default=np.nan)

In [ ]:
results_P['Label'].value_counts()

negative     51
neutral      38
undecided     9
positive      4
Name: Label, dtype: int64

In [ ]:
results_P.loc[results_P['Label']=='undecided']

,1,2,3,Label
Q4,0.0,0.5,0.5,undecided
Q41,0.5,0.0,0.5,undecided
Q53,0.0,0.5,0.5,undecided
Q63,0.2,0.4,0.4,undecided
Q65,0.0,0.5,0.5,undecided
Q70,0.0,0.5,0.5,undecided
Q76,0.0,0.5,0.5,undecided
Q91,0.2,0.4,0.4,undecided
Q99,0.0,0.5,0.5,undecided


In [ ]:
#Change undecided labels to either pos, neg, or neutral by use of an additional annotator
results_P.at['Q4','Label'] = 'neutral'
results_P.at['Q41','Label'] = 'negative'
results_P.at['Q53','Label'] = 'neutral'
results_P.at['Q63','Label'] = 'neutral'
results_P.at['Q65','Label'] = 'neutral'
results_P.at['Q70','Label'] = 'neutral'
results_P.at['Q76','Label'] = 'negative'
results_P.at['Q91','Label'] = 'neutral'
results_P.at['Q99','Label'] = 'neutral'

In [ ]:
results_P

,1,2,3,Label
Q1,0.1,0.0,0.9,negative
Q2,0.0,0.4,0.6,negative
Q3,0.0,0.8,0.2,neutral
Q4,0.0,0.5,0.5,neutral
Q5,0.1,0.0,0.9,negative
...,...,...,...,...
Q96,0.0,0.2,0.8,negative
Q97,0.0,0.2,0.8,negative
Q98,0.0,0.9,0.1,neutral
Q99,0.0,0.5,0.5,neutral


In [ ]:
#put tweet text in dataframe
results_P['text'] = results.T[0]

In [ ]:
results_P = results_P[['text','1','2','3','Label']]

In [ ]:
results_P['Label'].value_counts()

negative    53
neutral     45
positive     4
Name: Label, dtype: int64

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
results_P.to_csv('Survey_Results_Labeled_Dec.csv')
!cp Survey_Results_Labeled_Dec.csv "/content/drive/My Drive/Colab Notebooks"

In [ ]:
#DONT NEED THIS 
#Count the 1, 2, and 3s per row per question
#df1 = (results_T.apply(lambda x: x.value_counts(), axis=1)
         #.fillna(0)
         #.astype(int))

In [ ]:
#from statsmodels.stats.inter_rater import fleiss_kappa
#fleiss_kappa(df1,method='fleiss')

0.2583195210117539

In [ ]:
df2 = results.drop([0])

In [ ]:
!pip install krippendorff
import krippendorff

In [ ]:

df3 = df2.values.tolist()

In [ ]:
df4 = [[int(float(j)) for j in i] for i in df3]

In [ ]:
res = krippendorff.alpha(df4)

In [ ]:
res

0.24602453751648212